In [1]:
from tests.data_generation.stress_tests import run_evseq_stress_test

This is for stress-testing the evSeq code by passing in random inputs with known expected output. We then check to see if the evSeq outputs match the expected.

In [2]:
test_run = run_evseq_stress_test(True, False, 
                      keep_output = True,
                      seed = 2)

Working on tests for seed 2...
All Uncoupled tests passed for seed: 2
All Coupled tests passed for seed: 2


Working on tests for seed 3...


KeyboardInterrupt: 

In [15]:
test_run.config.variable_count

14

In [14]:
test_run.wells[8].global_dead

True

In [16]:
test_run.wells[8].n_dead_reads

10

In [17]:
test_run.wells[8].total_reads

98

In [18]:
test_run.wells[8].variants

In [19]:
test_run.wells[8].variants[0].total_counts

98

In [55]:
test_run.config.average_q_cutoff

16

In [51]:
import numpy as np
back_quals = np.concatenate((
            test_run.wells[8].refseq.rbc_qualities,
            test_run.wells[8].refseq.adapter_qualities_r,
            test_run.wells[8].refseq.frameshift_back_qualities,
            test_run.wells[8].refseq.primer_qualities_r
        ))

In [52]:
available_read_r = test_run.wells[8].config.readlength - len(back_quals)

In [53]:
total_r_quals = np.concatenate(
    (np.tile(back_quals, (98, 1)), np.flip(test_run.wells[8].variants[0].r_quals, axis = 1)),
    axis = 1)

In [74]:
np.argsort(test_run.wells[8].variants[0].r_quals[:, -available_read_r:].mean(axis = 1))

array([43, 52, 45, 61, 25, 84, 69, 32, 20, 19, 76, 60, 24, 59,  6, 83, 14,
        5, 56, 28, 11, 37, 65, 44, 81, 77,  3, 72, 87, 31, 46, 35, 41, 10,
       17, 36, 58,  8, 54, 79,  4, 86,  0, 62, 18, 38, 74,  2, 22, 13, 34,
       70,  1, 27, 21, 42, 55, 53, 12, 26, 63, 67, 40, 78, 39, 30, 57, 15,
       50, 68, 49, 29, 85, 71, 16, 23, 47, 33,  9, 48, 64, 80, 73, 66,  7,
       82, 75, 51, 95, 94, 93, 92, 88, 90, 89, 96, 91, 97])

In [75]:
np.mean([ord(character) - 33 for character in """GDE@D=F:FB<8=C?<BFB8:I:DE>F:BDH;D:89D98C>FIA@888DEE;A9<E:B:D;GCA;8(,&#.$+/,&./(00%/-++*(''#-%"0!',0**/.*,(&(!)+0'-!,',#%+0.!"*"0(.+)*#(.'*/&$*++.,.!-.-+%0-#!.0/.+&-$+0/.%'$)0"%(+)-/-%)!,%./0/'#"""])

16.22279792746114

To do:
1. Build code to test the "max" files coming out
4. Add on code to test for DEAD wells.

In [12]:
test_run.wells[385].variants[0].mutated_positions

array([ 1,  9, 13, 32, 35, 36, 43, 47, 95])

In [15]:
test_run.wells[385].global_dead

True

In [ ]:
test_run

In [4]:
import numpy as np
np.concatenate([
    np.array([1, 2, 3]),
    np.array([1, 2, 3]) + 1
]).dtype

dtype('int64')

In [5]:
test = np.array([1,2,3])
test[-10:]

array([1, 2, 3])